In [20]:
import pandas as pd
df=pd.read_csv('test_dataset.csv')
df.head()

,question,answer,response_generated
0,How much will you pay if my car is damaged?,Where damage to your car is covered under\nyou...,When your car is damaged and covered under you...
1,Can I use my car abroad?,"If you want to use your car abroad, your cover...","If you want to use your car abroad, your cover..."
2,Are my electric car’s charging cables covered?,Your home charger and charging cables are\ncon...,Your home charger and charging cables are cons...
3,"What does the ""Certificate of motor insurance""...",This document\nprovides evidence that you have...,"The ""Certificate of motor insurance"" document ..."
4,"What is an ""Automated car"" according to the do...",Your car where it is lawfully\ndriving itself ...,"An ""Automated car"" is defined as a car that is..."


In [33]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [22]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/Akshay Kamath/Desktop/pdfchat/policy-booklet-0923.pdf")
documents= loader.load()

In [31]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

Filename and doc_id are the same for all nodes.                 
Generating: 100%|██████████| 10/10 [05:08<00:00, 30.82s/it]


In [32]:
testset_df=testset.to_pandas()
testset_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How does a claim for an accident caused by an ...,[Section 6: Personal benefits\nPersonal benefi...,If you claim for an accident that isn’t your f...,simple,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
1,How does a claim for an accident caused by an ...,[Section 6: Personal benefits\nPersonal benefi...,If you claim for an accident that isn’t your f...,simple,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
2,"What types of losses, damages, or injuries are...",[Page 35Pressure waves\n 8We don’t cover any d...,"The insurance policy does not cover losses, da...",simple,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
3,What countries are included within the territo...,[Page 32Comprehensive Plus\nComp+\nYou’ll have...,"Andorra, Austria, Belgium, Bosnia and Herzegov...",simple,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
4,What are the conditions for accepting a Motor ...,[Page 26Making Motor Legal Cover claims\nAccep...,The conditions for accepting a Motor Legal Cov...,simple,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
5,What are the criteria for accepting a claim an...,[Page 26Making Motor Legal Cover claims\nAccep...,The criteria for accepting a claim are that th...,reasoning,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
6,If a policyholder has at least four years' wor...,[Page 30\nAbout Protected No Claim Discount\nT...,If a policyholder has at least four years' wor...,reasoning,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
7,What is the payment requirement for windscreen...,[Windscreen damage\nWe’ll help put things righ...,You'll need to pay an excess for each incident...,multi_context,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
8,What legal costs and expenses does the insuran...,[Section 1: Liability\nLiability to other peop...,The insurance policy covers reasonable legal c...,multi_context,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True
9,What are some situations or claims that are no...,[Page 25Section 7: Motor Legal Cover\nYou’re n...,Some situations or claims that are not covered...,simple,[{'source': 'C:/Users/Akshay Kamath/Desktop/pd...,True


In [15]:
from datasets import load_dataset
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

In [30]:
from ragas import evaluate

result = evaluate(
    testset_df,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

df = result.to_pandas()
print(df)

AttributeError: 'DataFrame' object has no attribute 'rename_columns'